In [ ]:
import cv2
import numpy as np


def calculate_distances(keypoints):
    distances = []
    for i in range(len(keypoints)):
        for j in range(i+1, len(keypoints)):
            pt1 = np.array(keypoints[i].pt)
            pt2 = np.array(keypoints[j].pt)
            distance = np.linalg.norm(pt1 - pt2)
            distances.append(distance)
    return distances


def find_smallest_distance(distances):
    if distances:
        return min(distances)
    return 0


def estimate_positions(center, radius, smallest_distance):
    num_positions = int(2 * np.pi * radius / smallest_distance)
    angles = np.linspace(0, 2 * np.pi, num_positions, endpoint=False)
    expected_positions = []
    for angle in angles:
        x = center[0] + radius * np.cos(angle)
        y = center[1] + radius * np.sin(angle)
        expected_positions.append((x, y))
    return expected_positions


image = cv2.imread(r"C:\Users\ayush\Downloads\projectidentifymissingholes\18_8h.jpg", cv2.IMREAD_GRAYSCALE)


params = cv2.SimpleBlobDetector_Params()
params.minThreshold = 10
params.maxThreshold = 200
params.filterByArea = True
params.minArea = 100  
params.maxArea = 500  
params.filterByCircularity = False 
params.filterByConvexity = True
params.minConvexity = 0.2 
params.filterByInertia = False  
params.filterByColor = True
params.blobColor = 255  


detector = cv2.SimpleBlobDetector_create(params)


keypoints = detector.detect(image)
print(keypoints)


if keypoints:
    pts = np.array([kp.pt for kp in keypoints], dtype=np.float32)
    (x, y), radius = cv2.minEnclosingCircle(pts)
    center = (int(x), int(y))
    radius = int(radius)
else:
   
    center = (image.shape[1] // 2, image.shape[0] // 2)
    radius = min(image.shape[1], image.shape[0]) // 2 - 10


distances = calculate_distances(keypoints)


smallest_distance = find_smallest_distance(distances)


expected_positions = estimate_positions(center, radius, smallest_distance)


detected_positions = np.array([kp.pt for kp in keypoints])


im_with_keypoints = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)
im_with_keypoints = cv2.drawKeypoints(im_with_keypoints, keypoints, np.array([]), (0, 0, 255),
                                      cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)


cv2.circle(im_with_keypoints, center, radius, (0, 255, 0), 2)


for ep in expected_positions:
    cv2.circle(im_with_keypoints, (int(ep[0]), int(ep[1])), 10, (255, 0, 0), 2)  # Smaller radius of 5


cv2.imshow("Image with keypoints and expected positions", im_with_keypoints)
cv2.waitKey(0)
cv2.destroyAllWindows()


output_path = r'C:\Users\ayush\Downloads\missingholes\result_image_with_keypoints.jpg'
cv2.imwrite(output_path, im_with_keypoints)


number_of_blobs = len(keypoints)
print(f"Number of blobs detected: {number_of_blobs}")
print(f"Number of total holes: {len(expected_positions)}")
print(f'Number of missing holes: {len(expected_positions) - number_of_blobs}')


In [7]:
import cv2
import numpy as np


def calculate_distances(keypoints):
    distances = []
    for i in range(len(keypoints)):
        for j in range(i + 1, len(keypoints)):
            pt1 = np.array(keypoints[i].pt)
            pt2 = np.array(keypoints[j].pt)
            distance = np.linalg.norm(pt1 - pt2)
            distances.append(distance)
    return distances


def find_smallest_distance(distances):
    if distances:
        return min(distances)
    return 0

# Function to estimate expected positions of holes based on the ellipse
def estimate_ellipse_positions(center, axes, angle, smallest_distance):
    # Calculate the perimeter of the ellipse using Ramanujan's approximation
    a, b = axes[0] / 2, axes[1] / 2  # Divide by 2 as axes in ellipse are diameters
    h = ((a - b)**2) / ((a + b)**2)
    ellipse_perimeter = np.pi * (a + b) * (1 + (3 * h) / (10 + np.sqrt(4 - 3 * h)))

    # Calculate the number of expected positions along the ellipse
    num_positions = int(ellipse_perimeter / smallest_distance)

    # Generate the expected positions along the ellipse perimeter
    angles = np.linspace(0, 2 * np.pi, num_positions, endpoint=False)
    expected_positions = []
    cos_angle = np.cos(np.radians(angle))
    sin_angle = np.sin(np.radians(angle))
    for theta in angles:
        x = center[0] + a * np.cos(theta) * cos_angle - b * np.sin(theta) * sin_angle
        y = center[1] + a * np.cos(theta) * sin_angle + b * np.sin(theta) * cos_angle
        expected_positions.append((x, y))
    return expected_positions


image = cv2.imread(r"C:\Users\ayush\Downloads\projectidentifymissingholes\18_8h.jpg", cv2.IMREAD_GRAYSCALE)


params = cv2.SimpleBlobDetector_Params()
params.minThreshold = 10
params.maxThreshold = 200
params.filterByArea = True
params.minArea = 100  
params.maxArea = 500  
params.filterByCircularity = False 
params.filterByConvexity = True
params.minConvexity = 0.2  
params.filterByInertia = False 
params.filterByColor = True
params.blobColor = 255  


detector = cv2.SimpleBlobDetector_create(params)


keypoints = detector.detect(image)


if keypoints:
    pts = np.array([kp.pt for kp in keypoints], dtype=np.float32)

    ellipse = cv2.fitEllipse(pts)
else:
    # Handle case where no keypoints (blobs) are detected
    center = (image.shape[1] // 2, image.shape[0] // 2)
    ellipse = ((center[0], center[1]), (0, 0), 0)


im_with_keypoints = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)
im_with_keypoints = cv2.drawKeypoints(im_with_keypoints, keypoints, np.array([]), (0, 0, 255),
                                      cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)


if keypoints:
    cv2.ellipse(im_with_keypoints, ellipse, (255, 0, 0), 2)


distances = calculate_distances(keypoints)


smallest_distance = find_smallest_distance(distances)


if keypoints:
    center = ellipse[0]
    axes = ellipse[1]
    angle = ellipse[2]
    expected_positions = estimate_ellipse_positions(center, axes, angle, smallest_distance)
else:
    expected_positions = []


for pos in expected_positions:
    cv2.circle(im_with_keypoints, (int(pos[0]), int(pos[1])), 5, (0, 255, 0), -1)


cv2.imshow("Image with keypoints, fitted ellipse, and expected positions", im_with_keypoints)
cv2.waitKey(0)
cv2.destroyAllWindows()


output_path = r"C:\Users\ayush\Downloads\missingholes\result_image_with_keypoints_ellipse_and_positions.jpg"
cv2.imwrite(output_path, im_with_keypoints)


number_of_blobs = len(keypoints)
print(f"Number of blobs detected: {number_of_blobs}")
print(f"Number of total holes: {len(expected_positions)}")
print(f'Number of missing holes: {len(expected_positions) - number_of_blobs}')


Number of blobs detected: 8
Number of total holes: 8
Number of missing holes: 0
